In [ ]:
import pandas as pd
import re

# เส้นทางไฟล์ต้นฉบับ
file_path = '/content/ข่าว-clean3.txt'

# อ่านไฟล์เข้าเป็น list ของบรรทัด
with open(file_path, 'r', encoding='utf-8') as f:
    lines = f.readlines()

data = []
current_ticker = None
i = 0

while i < len(lines):
    line = lines[i].strip()

    # ตรวจสอบบรรทัดที่ขึ้นต้นด้วย "หุ้น " เพื่อกำหนด ticker ปัจจุบัน
    if line.startswith('หุ้น '):
        # ตัดเอาเฉพาะตัวย่อหุ้น หลังคำว่า "หุ้น "
        current_ticker = line.split()[1]
        i += 1
        continue

    # ตรวจสอบบรรทัดที่เป็นจุดเริ่มต้นของข่าว (เช่น "1. หัวข้อข่าว: …")
    match_headline = re.match(r'\d+\.\s*หัวข้อข่าว:\s*(.*)', line)
    if match_headline:
        headline = match_headline.group(1).strip()

        # บรรทัดถัดไปควรเป็นวันที่เขียนข่าว, เนื้อหาข่าว, วิเคราะห์, และ เหตุผล
        date_line = lines[i + 1].strip()
        content_line = lines[i + 2].strip()
        sentiment_line = lines[i + 3].strip()
        reason_line = lines[i + 4].strip()

        # ใช้ regex ดึงข้อมูลแต่ละส่วนออกมา
        date_match = re.match(r'วันที่เขียนข่าว:\s*(.*)', date_line)
        content_match = re.match(r'เนื้อหาข่าว:\s*(.*)', content_line)
        sentiment_match = re.match(r'วิเคราะห์:\s*(.*)', sentiment_line)
        reason_match = re.match(r'เหตุผล:\s*(.*)', reason_line)

        date = date_match.group(1).strip() if date_match else ''
        content = content_match.group(1).strip() if content_match else ''
        sentiment = sentiment_match.group(1).strip() if sentiment_match else ''
        reason = reason_match.group(1).strip() if reason_match else ''

        # เก็บเป็น dict แล้ว append เข้า list
        data.append({
            'ตัวย่อหุ้น': current_ticker,
            'หัวข้อข่าว': headline,
            'วันที่เขียนข่าว': date,
            'เนื้อหาข่าว': content,
            'วิเคราะห์': sentiment,
            'เหตุผล': reason
        })

        # กระโดดข้าม 5 บรรทัด (หัวข้อข่าว + 4 บรรทัดถัดมา)
        i += 5
    else:
        i += 1

# นำ list ของ dict มาสร้างเป็น DataFrame
df = pd.DataFrame(
    data,
    columns=[
        'ตัวย่อหุ้น', 'หัวข้อข่าว', 'วันที่เขียนข่าว',
        'เนื้อหาข่าว', 'วิเคราะห์', 'เหตุผล'
    ]
)

# บันทึกเป็นไฟล์ CSV (UTF-8 พร้อม BOM เพื่อให้เปิดใน Excel ได้ถูกต้อง)
csv_path = 'news.csv'
df.to_csv(csv_path, index=False, encoding='utf-8-sig')

print(f"สร้างไฟล์ CSV เรียบร้อยแล้ว ที่: {csv_path}")


สร้างไฟล์ CSV เรียบร้อยแล้ว ที่: news.csv


In [7]:
import pandas as pd

# โหลดไฟล์ CSV ที่ได้จากการแปลง
df = pd.read_csv("/content/news.csv")  # เปลี่ยนชื่อไฟล์หากใช้ชื่ออื่น

# แสดงข้อมูล 5 แถวแรก
print("\nข้อมูล 5 แถวแรก:")
df.head()



ข้อมูล 5 แถวแรก:


,ตัวย่อหุ้น,หัวข้อข่าว,วันที่เขียนข่าว,เนื้อหาข่าว,วิเคราะห์,เหตุผล
0,WHA,สรุป pp Day : WHA จ่อปิด 2 ดีลใหญ่ ช่วง Q2Q3 น...,30/05/2025,ข้อมูลจาก pp Day ระบุว่า WHA กำลังเจรจาปิดดีลล...,บวก,การมีลูกค้าดาต้าเซ็นเตอร์ต่อคิวรอทำสัญญาถึงกว่...
1,WHA,สะกิดต่อมหุ้น อัปเดต ประจำวันที่ 30 พ.ค. 2568:...,30/05/2025,ในการอัปเดตตลาดหุ้นประจำวัน WHA ปรับตัวขึ้นกว่...,บวก,การเปลี่ยนแปลงเชิงบวกในกรอบนโยบายภาษีสหรัฐฯ จะ...
2,WHA,WHA จับตาผลภาษีสหรัฐฯ ชัดเจนในครึ่งปีหลัง ก่อน...,02/06/2025,สำนักข่าวรายงานว่า WHA กำลังจับตาการกำหนดอัตรา...,บวก,การจับตาภาษีนำเข้าชัดเจนจะช่วยให้ WHA สามารถทบ...
3,WHA,"WHA กำไร Q1/68 ที่ 2,075.47 ลบ. โต 52.06% ขายท...",12/05/2025,"WHA รายงานกำไรสุทธิไตรมาส 1/2568 ที่ 2,075.47 ...",บวก,กำไรสุทธิที่เติบโตสูงถึง 52.06% ในไตรมาสแรกเป็...
4,WHA,หุ้นนิคมฯ ไตรมาส 2/68 ยอดขายแข็งแกร่ง ด้านภาษี...,02/06/2025,บทวิเคราะห์จาก สำนักข่าวบางกอกบิสเนส รายงานว่า...,บวก,การเข้าร่วมเจรจาผ่อนปรนภาษีช่วยลดต้นทุนตลอดห่ว...


In [15]:
print("\nข้อมูลเบื้องต้นของ DataFrame:")
df.info()


ข้อมูลเบื้องต้นของ DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1117 entries, 0 to 1116
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   ตัวย่อหุ้น       1117 non-null   object
 1   หัวข้อข่าว       1117 non-null   object
 2   วันที่เขียนข่าว  433 non-null    object
 3   เนื้อหาข่าว      1116 non-null   object
 4   วิเคราะห์        1010 non-null   object
 5   เหตุผล           977 non-null    object
dtypes: object(6)
memory usage: 52.5+ KB


In [16]:
print("\nสถิติเชิงพรรณนา:")
print(df.describe())


สถิติเชิงพรรณนา:
       ตัวย่อหุ้น                                   หัวข้อข่าว  \
count        1117                                         1117   
unique        206                                         1089   
top           PSH  DELTA จ่ายปันผล 1.10 บาทต่อหุ้น XD 05/05/68   
freq           12                                            2   

       วันที่เขียนข่าว                                        เนื้อหาข่าว  \
count              433                                               1116   
unique             295                                               1094   
top         15/05/2025  DMT แจ้งตลาดหลักทรัพย์ฯ ว่าที่ประชุมบอร์ดมีมติ...   
freq                12                                                  2   

       วิเคราะห์                                             เหตุผล  
count       1010                                                977  
unique        25                                                970  
top          บวก  ธุรกิจที่เกี่ยวข้องกับสุขภาพและการแพท

In [17]:
print("\nจำนวนค่าที่ขาดหายในแต่ละคอลัมน์:")
df.isnull().sum()


จำนวนค่าที่ขาดหายในแต่ละคอลัมน์:


,0
ตัวย่อหุ้น,0
หัวข้อข่าว,0
วันที่เขียนข่าว,684
เนื้อหาข่าว,1
วิเคราะห์,107
เหตุผล,140


In [18]:
print("\nค่าที่ไม่ซ้ำกันในคอลัมน์ 'วิเคราะห์':")
df['วิเคราะห์'].unique()


ค่าที่ไม่ซ้ำกันในคอลัมน์ 'วิเคราะห์':


array(['บวก', 'กลาง', 'ลบ', nan, 'กลาง–ลบ', 'บวก–กลาง', 'กลาง–บวก',
       'กลาง–ลบเล็กน้อย', 'บวกระยะยาว', 'กลางถึงลบเล็กน้อย',
       'กลางถึงบวกเล็กน้อย', 'บวกเล็กน้อย', 'ลบเล็กน้อย', 'เชิงบวก',
       'เชิงลบ', 'ลบ–กลาง', 'บวก (เชิงกลยุทธ์)', 'บวก (เชิง ESG)',
       'กลาง–บวก (เชิง ESG)', 'กลางถึงลบ', 'กลางถึงบวก', '“ลบ”', '“บวก”',
       '“กลาง”', 'บวก (เชิงเทคนิคและ Sentiment)',
       'บวกเล็กน้อย (เชิง Sentiment ฤดูกาล)'], dtype=object)

In [19]:
print("\nจำนวนข่าวตามประเภท 'วิเคราะห์':")
df['วิเคราะห์'].value_counts()


จำนวนข่าวตามประเภท 'วิเคราะห์':


,count
วิเคราะห์,
บวก,570
กลาง,203
ลบ,125
“บวก”,35
เชิงบวก,17
“กลาง”,11
“ลบ”,9
กลาง–บวก,8
กลางถึงบวก,6


In [24]:
# กำหนด mapping สำหรับการจัดกลุ่มคำวิเคราะห์
sentiment_mapping = {
    'บวก': ['บวก', 'เชิงบวก', 'บวก (เชิง ESG)', 'บวกเล็กน้อย', 'บวกระยะยาว', 'บวก (เชิงกลยุทธ์)', 'บวกเล็กน้อย (เชิง Sentiment ฤดูกาล)'],
    'กลาง': ['กลาง', 'กลาง–บวก', 'กลางถึงบวก', 'บวก–กลาง', 'กลาง–ลบเล็กน้อย', 'กลางถึงบวกเล็กน้อย', 'เชิงลบ', 'กลาง–บวก (เชิง ESG)', 'บวก (เชิงเทคนิคและ Sentiment)'],
    'ลบ': ['ลบ', 'กลาง–ลบ', 'ลบเล็กน้อย', 'กลางถึงลบเล็กน้อย', 'กลางถึงลบ', 'ลบ–กลาง']
}

# สร้างฟังก์ชันสำหรับแปลงค่าในคอลัมน์ 'วิเคราะห์'
def categorize_sentiment(sentiment):
    if pd.isna(sentiment): # จัดการค่า NaN
        return 'ไม่ระบุ'
    for category, keywords in sentiment_mapping.items():
        if sentiment.strip() in keywords:
            return category
    return 'อื่นๆ' # กรณีที่ไม่ตรงกับเงื่อนไขใดๆ

# ใช้ apply เพื่อแปลงค่าในคอลัมน์ 'วิเคราะห์'
df['วิเคราะห์_grouped'] = df['วิเคราะห์'].apply(categorize_sentiment)

# แสดงผลลัพธ์หลังการแปลง
print("\nข้อมูล 5 แถวแรก พร้อมคอลัมน์ 'วิเคราะห์_grouped':")
print(df[['วิเคราะห์', 'วิเคราะห์_grouped']].head())

print("\nจำนวนข่าวตามประเภท 'วิเคราะห์_grouped':")
df['วิเคราะห์_grouped'].value_counts()


ข้อมูล 5 แถวแรก พร้อมคอลัมน์ 'วิเคราะห์_grouped':
  วิเคราะห์ วิเคราะห์_grouped
0       บวก               บวก
1       บวก               บวก
2       บวก               บวก
3       บวก               บวก
4       บวก               บวก

จำนวนข่าวตามประเภท 'วิเคราะห์_grouped':


,count
วิเคราะห์_grouped,
บวก,594
กลาง,223
ลบ,138
ไม่ระบุ,107
อื่นๆ,55


In [26]:
df['วิเคราะห์'] = df['วิเคราะห์_grouped']
df['วิเคราะห์']

,วิเคราะห์
0,บวก
1,บวก
2,บวก
3,บวก
4,บวก
...,...
1112,บวก
1113,กลาง
1114,บวก
1115,กลาง


In [27]:
# นับจำนวนข่าวของแต่ละตัวย่อหุ้น
ticker_counts = df['ตัวย่อหุ้น'].value_counts()

# แสดงผล
print("จำนวนข่าวต่อแต่ละตัวย่อหุ้น:")
print(ticker_counts)


จำนวนข่าวต่อแต่ละตัวย่อหุ้น:
ตัวย่อหุ้น
PSH      12
DELTA    10
ASIAN    10
ASK      10
DITT     10
         ..
ILINK     5
AGE       4
BAFS      4
KCE       4
CM        3
Name: count, Length: 206, dtype: int64


In [28]:
# คัดเฉพาะตัวย่อหุ้นที่มีข่าวมากกว่า 5 ข่าว
tickers_over_5 = ticker_counts[ticker_counts > 5]

# แสดงผล
print("ตัวย่อหุ้นที่มีจำนวนข่าวมากกว่า 5 ข่าว:")
print(tickers_over_5)


ตัวย่อหุ้นที่มีจำนวนข่าวมากกว่า 5 ข่าว:
ตัวย่อหุ้น
PSH       12
DELTA     10
ASIAN     10
ASK       10
DITT      10
DMT       10
DEMC      10
TFG       10
TGE       10
TEGH      10
BGC       10
STECN     10
BGRIM     10
S         10
STA       10
STGT      10
SUN       10
TFMAMA    10
Name: count, dtype: int64


In [29]:
# คัดเฉพาะตัวย่อหุ้นที่มีข่าวน้อยกว่า 5 ข่าว
tickers_over_5 = ticker_counts[ticker_counts < 5]

# แสดงผล
print("ตัวย่อหุ้นที่มีจำนวนข่าวน้อยกว่า 5 ข่าว:")
print(tickers_over_5)


ตัวย่อหุ้นที่มีจำนวนข่าวน้อยกว่า 5 ข่าว:
ตัวย่อหุ้น
AGE     4
BAFS    4
KCE     4
CM      3
Name: count, dtype: int64


In [30]:
# กรองข้อมูลเฉพาะหุ้น PT
df_pt = df[df['ตัวย่อหุ้น'] == 'SUN']

# แสดงข้อมูลทั้งหมดของหุ้น PT
print("\nข้อมูลข่าวสารของหุ้น PT:")
if not df_pt.empty:
    print(df_pt.to_string()) # ใช้ to_string() เพื่อแสดงผลทั้งหมดโดยไม่ตัดบรรทัด
else:
    print("ไม่พบข้อมูลข่าวสารสำหรับหุ้น PT")


ข้อมูลข่าวสารของหุ้น PT:
    ตัวย่อหุ้น                                                                                              หัวข้อข่าว วันที่เขียนข่าว                                                                                                                                                                                                                                                                                                                                                                                    เนื้อหาข่าว วิเคราะห์                                                                                                                                                                                        เหตุผล วิเคราะห์_grouped
631        SUN    SUN ลงทุนตั้งร่วมทุน “ไทยยูเนี่ยน–UFLEX” ขยายไลน์ผลิตบรรจุภัณฑ์อาหารถุงปรุงสำเร็จ มูลค่า 200 ล้านบาท             NaN                                                                                                        

In [32]:
# สร้าง DataFrame ใหม่ที่ไม่รวมคอลัมน์ 'วิเคราะห์_grouped'
df_to_save = df.drop(columns=['วิเคราะห์_grouped'])

# บันทึก DataFrame ใหม่เป็นไฟล์ CSV
csv_path_new = 'news.csv'
df_to_save.to_csv(csv_path_new, index=False, encoding='utf-8-sig')

print(f"สร้างไฟล์ CSV ใหม่โดยไม่มีคอลัมน์ 'วิเคราะห์_grouped' เรียบร้อยแล้ว ที่: {csv_path_new}")
print("\nข้อมูล 5 แถวแรกของ DataFrame ที่จะบันทึก:")
df_to_save.head()

สร้างไฟล์ CSV ใหม่โดยไม่มีคอลัมน์ 'วิเคราะห์_grouped' เรียบร้อยแล้ว ที่: news.csv

ข้อมูล 5 แถวแรกของ DataFrame ที่จะบันทึก:


,ตัวย่อหุ้น,หัวข้อข่าว,วันที่เขียนข่าว,เนื้อหาข่าว,วิเคราะห์,เหตุผล
0,WHA,สรุป pp Day : WHA จ่อปิด 2 ดีลใหญ่ ช่วง Q2Q3 น...,30/05/2025,ข้อมูลจาก pp Day ระบุว่า WHA กำลังเจรจาปิดดีลล...,บวก,การมีลูกค้าดาต้าเซ็นเตอร์ต่อคิวรอทำสัญญาถึงกว่...
1,WHA,สะกิดต่อมหุ้น อัปเดต ประจำวันที่ 30 พ.ค. 2568:...,30/05/2025,ในการอัปเดตตลาดหุ้นประจำวัน WHA ปรับตัวขึ้นกว่...,บวก,การเปลี่ยนแปลงเชิงบวกในกรอบนโยบายภาษีสหรัฐฯ จะ...
2,WHA,WHA จับตาผลภาษีสหรัฐฯ ชัดเจนในครึ่งปีหลัง ก่อน...,02/06/2025,สำนักข่าวรายงานว่า WHA กำลังจับตาการกำหนดอัตรา...,บวก,การจับตาภาษีนำเข้าชัดเจนจะช่วยให้ WHA สามารถทบ...
3,WHA,"WHA กำไร Q1/68 ที่ 2,075.47 ลบ. โต 52.06% ขายท...",12/05/2025,"WHA รายงานกำไรสุทธิไตรมาส 1/2568 ที่ 2,075.47 ...",บวก,กำไรสุทธิที่เติบโตสูงถึง 52.06% ในไตรมาสแรกเป็...
4,WHA,หุ้นนิคมฯ ไตรมาส 2/68 ยอดขายแข็งแกร่ง ด้านภาษี...,02/06/2025,บทวิเคราะห์จาก สำนักข่าวบางกอกบิสเนส รายงานว่า...,บวก,การเข้าร่วมเจรจาผ่อนปรนภาษีช่วยลดต้นทุนตลอดห่ว...
